# Geocoding addresses using the geoadmin API and Python

## Libraries and settings

In [49]:
# Libraries
import os
import requests
import json
import urllib
import fnmatch
import folium
import platform
import pandas as pd
import geopandas as gpd
from IPython.display import clear_output
from folium.features import GeoJson

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Load the CSV file

In [50]:
df = pd.read_csv('/Users/erxhanganiu/ADSProjekt/AppliedDataScience/Backups/immoscout24.csv', delimiter=';')

# Define base url for geocoding

In [51]:
base_url = "https://api3.geo.admin.ch/rest/services/api/SearchServer?"

# Geocode list of addresses

### Geocoding addresses using the geoadmin API

In [57]:
# Define base url
base_url= "https://api3.geo.admin.ch/rest/services/api/SearchServer?"

# Geocode list of adresses
geolocation = []
n = 1
for i in df['Address'].astype(str):
    print('Geocoding address', 
          n, 
          'out of', 
          len(df['Address']), 
          ':', 
          i)
    n += 1
    clear_output(wait=True)
    try:
        # Set up search parameters - address, origins, and type
        parameters = {
            'searchText': i,
            'type': 'locations',
            'origins': 'address'
        }
        r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")
        data = json.loads(r.content)
        # Extract latitude and longitude from response
        # Add the code to extract lat, lon and store in geolocation list
    except Exception as e:
        print("Error geocoding:", e)

# Write lat and lon to df
df_loc = pd.DataFrame(geolocation, 
                      columns=("lat", "lon"), 
                      index=range(len(df['address_raw'])))
df['lat'] = df_loc['lat']
df['lon'] = df_loc['lon']
df.head(5)

KeyError: 'address_raw'

### Read polygon-map with municipalities of the canton of Zuerich

In [53]:
# Polygonmap als .json-File (WGS84)
polys = gpd.read_file("/Users/erxhanganiu/ADSProjekt/AppliedDataScience/gadm41_CHE_3.json")
print(type(polys))
polys.head(5)

<class 'geopandas.geodataframe.GeoDataFrame'>


,GID_3,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,NL_NAME_2,NAME_3,VARNAME_3,NL_NAME_3,TYPE_3,ENGTYPE_3,CC_3,HASC_3,geometry
0,CHE.1.1.1_1,CHE,Switzerland,CHE.1_1,Aargau,NA,CHE.1.1_1,Aarau,NA,Aarau,NA,NA,Commune,Municipality,NA,NA,"MULTIPOLYGON (((8.07110 47.40240, 8.06550 47.3..."
1,CHE.1.1.2_1,CHE,Switzerland,CHE.1_1,Aargau,NA,CHE.1.1_1,Aarau,NA,Biberstein,NA,NA,Commune,Municipality,NA,NA,"MULTIPOLYGON (((8.06960 47.41470, 8.06720 47.4..."
2,CHE.1.1.3_1,CHE,Switzerland,CHE.1_1,Aargau,NA,CHE.1.1_1,Aarau,NA,Buchs,NA,NA,Commune,Municipality,NA,NA,"MULTIPOLYGON (((8.06320 47.39300, 8.06520 47.3..."
3,CHE.1.1.4_1,CHE,Switzerland,CHE.1_1,Aargau,NA,CHE.1.1_1,Aarau,NA,Densbüren,NA,NA,Commune,Municipality,NA,NA,"MULTIPOLYGON (((8.06670 47.43720, 8.06190 47.4..."
4,CHE.1.1.5_1,CHE,Switzerland,CHE.1_1,Aargau,NA,CHE.1.1_1,Aarau,NA,Erlinsbach,NA,NA,Commune,Municipality,NA,NA,"MULTIPOLYGON (((8.03260 47.40540, 8.02840 47.4..."


### Plot map

In [54]:
# Überprüfen der Spaltennamen
print(df.columns)

Index(['Address', 'Title', 'Description', 'Rooms', 'Living Space (sqm)',
       'Price'],
      dtype='object')


In [56]:
# Initialisierung der Map
m = folium.Map(location=[47.44, 8.65], zoom_start=10)

# Map settings
folium.Choropleth(
    geo_data=polys,
    name='polys',
    fill_color='greenyellow'
).add_to(m)

# Add lat/lon of addresses
df_sub = df.dropna()
for i in range(0, len(df_sub)):
    folium.Marker(location=(df_sub.iloc[i]['lat'], 
                            df_sub.iloc[i]['lon']), 
                  popup=df_sub.iloc[i]['address_raw']).add_to(m)

# Layer control
folium.LayerControl().add_to(m)

# Plot map
m

KeyError: 'lat'

### Intersect municipality polygon-map with lat and lon (point-in-polygon intersection)

### Save data to file

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')